# Development of AI in scientific research

Artificial Intelligence (AI) has emerged as one of the most talked-about topics in recent years. Breakthroughs in technologies such as machine learning(ML), large language models (LLM), and computer vision (CV) have not only transformed how we live and work but have also  created a significant buzz in the economy. Major tech companies like Google, Microsoft or NVIDIA have achieved tremendous profits and remarkable stock market growth through their investments and research in AI. The innovative potential of AI is widely regarded as a catalyst for economic growth and technological progress.

While the societal and economic enthusiasm for AI is undeniable, this analysis aims to investigate whether a comparable trend can be observed in the scientific community. This analysis examines publication data from the past three decades to identify patterns in research activity and explore whether the increasing prominence of AI is reflected in scientific output. The focus is directed toward publications in key disciplines such as computer science, mathematics, quantitative finance, and statistics, which provide the foundational framework for advancements in AI.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Override default plotly theme to white background with gridlines
pio.templates.default = "plotly_white"

In [2]:
TITLE_SIZE = 25
TITLE_X_POS = 0.5

## Data

The data for this analysis was sourced from arXiv.org, a repository for scientific preprints. To investigate the development and trends in Artificial Intelligence research, publications from 1994 to the end of 2024 were queried. The focus was set on categories that are highly relevant to the theoretical and practical advancements in AI. These categories were carefully selected to ensure coverage of core disciplines such as computer science, mathematics, quantitative finance, and statistics, which are central to the development of AI methodologies and applications.

The selected categories reflect a comprehensive view of AI research, ranging from foundational theories to applied methodologies. This approach also highlights the interplay between traditional mathematical and statistical methods and emerging AI driven techniques. The inclusion criteria were designed to capture the evolution of AI research over time, while broader, less directly related domains, such as economics, electrical engineering, physics, and quantitative biology were  excluded. An exception was made for quantitative finance due to its role in applying methods such as Long Short-Term Memory (LSTM) neural networks, which have significant applications in financial time series modeling and other AI-related fields.

The analysis specifically includes publications from the following domains:
| Domain                | Category   | Description                                   |
|-----------------------|------------|-----------------------------------------------|
| **Computer Science**  | cs.AI      | Artificial Intelligence                      |
|                       | cs.CC      | Computational Complexity                     |
|                       | cs.CL      | Computation and Language                     |
|                       | cs.CV      | Computer Vision and Pattern Recognition      |
|                       | cs.DM      | Discrete Mathematics                         |
|                       | cs.HC      | Human-Computer Interaction                   |
|                       | cs.LG      | Machine Learning                             |
|                       | cs.MA      | Multiagent Systems                           |
|                       | cs.NE      | Neural and Evolutionary Computing            |
|                       | cs.RO      | Robotics                                     |
|                       | cs.SD      | Sound                                        |
| **Mathematics**       | math.DS    | Dynamical Systems                            |
|                       | math.OC    | Optimization and Control                     |
|                       | math.PR    | Probability                                  |
|                       | math.QA    | Quantum Algebra                              |
|                       | math.ST    | Statistics                                   |
| **Quantitative Finance** | q-fin.CP   | Computational Finance                        |
|                       | q-fin.MF   | Mathematical Finance                         |
|                       | q-fin.ST   | Statistical Finance                          |
| **Statistics**        | stat.ML    | Machine Learning in Statistics               |
|                       | stat.TH    | Theoretical Statistics                       |


In [3]:
CATEGORIES = [
    "cs.lg",
    "cs.ai",
    "cs.cc",
    "math.st",
    "math.oc",
    "math.pr",
    "stat.ml",
    "cs.cv",
    "cs.cl",
    "cs.ne",
    "cs.ro",
    "math.ds",
    "cs.hc",
    "cs.dm",
    "cs.ma",
    "cs.sd",
    "math.qa",
    "q-fin.st",
    "q-fin.mf",
    "q-fin.cp",
    "stat.th",
]

ARXIV_CATEGORIES_MAPPING = {
    "cs.LG": "Machine Learning (Computer Science)",
    "cs.AI": "Artificial Intelligence",
    "cs.CC": "Computational Complexity",
    "math.ST": "Statistics Theory (Mathematics)",
    "math.OC": "Optimization and Control",
    "math.PR": "Probability",
    "stat.ML": "Machine Learning (Statistics)",
    "cs.CV": "Computer Vision and Pattern Recognition",
    "cs.CL": "Computation and Language",
    "cs.NE": "Neural and Evolutionary Computing",
    "cs.RO": "Robotics",
    "math.DS": "Dynamical Systems",
    "cs.HC": "Human-Computer Interaction",
    "cs.DM": "Discrete Mathematics",
    "cs.MA": "Multiagent Systems",
    "cs.SD": "Sound",
    "math.QA": "Quantum Algebra",
    "q-fin.ST": "Statistical Finance",
    "q-fin.MF": "Mathematical Finance",
    "q-fin.CP": "Computational Finance",
    "stat.TH": "Theoretical Statistics"
}


In [ ]:
# Load Data
df_complete = pd.read_csv("arxiv_papers.csv")
df_complete

In [24]:
# Convert date from string to datetime object
df_complete["published_date"] = pd.to_datetime(df_complete["published_date"])

## Helper Functions

In [25]:
def prepare_data(df, period, also_group_categories=False):
    """
    Prepares the data for visualization by adding a period column based on the specified frequency
    """
    df = df.copy(deep=True)

    # Add period column based on specified frequency
    if period.lower() == "y":
        df["period"] = df["published_date"].dt.to_period(period).dt.year
    else:    
        df["period"] = df["published_date"].dt.to_period(period).dt.to_timestamp()

    if also_group_categories:
        # Convert comma-separated string of categories to python list
        df['categories'] = df['categories'].str.lower().str.split(',')

        # Explode the categories into separate rows
        df = df.explode("categories")
        df["categories"] = df["categories"].str.strip()

        # Filter rows to only keep the ones matching a specified category
        df = df[df['categories'].isin(CATEGORIES)]

        # Group the data by categories and period and count occurrences
        df = df.groupby(['categories', 'period']).size().reset_index(name='count')
    else:
        # Group data by just the period and count occurrences
        df = df.groupby("period").size().reset_index(name="count")
    
    return df


In [26]:
def generate_color_map(categories):
    """
    Generate a consistent color map for categories to ensure that each category is assigned a unique color.

    This is done for the visualization of the line charts that are side-by-side so that the same category appears with the same color in both plots.
    """
    return {category: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
            for i, category in enumerate(categories)}


## Visualization

The dataset analyzed includes a unique identifier for each publication, referred to as *paper_id*. This identifier features a versioning system, where the suffix *v1* denotes the initial release of a paper. Later versions, such as *v2*, *v3* and beyond, typically reflect corrections, refinements or improvements to the original work. While these subsequent versions address important refinements, they generally do not introduce groundbreaking ideas. Therefore, this analysis focuses primarily on the initial publications (*v1*), as they are considered the most significant in contributing to scientific progress.

The structure of the analysis is designed to provide a clear and comprehensive understanding of the trends in AI and ML related research. It begins with a broad overview of publication activity, including all versions, to illustrate the overall growth in the field. The focus then narrows to initial publications, highlighting their foundational role in advancing knowledge. The next section explores category specific trends, offering a detailed examination of key areas such as machine learning, computer vision, and statistics. Finally, a relative analysis is presented to highlight the proportional impact of individual categories over time, offering insights into their evolving significance.

### General Development of Publications in AI and ML related fields

In [27]:
def plot_general_development(df, title, period):
    """
    Plots the number of publications per period on both linear and logarithmic scales side-by-side.
    """
    papers_per_period = prepare_data(df, period)

    # Create a scatter plot with a linear scale
    # To enable the ols trendline to depict an exponential trend log_y is set to True only for the trendline
    fig_linear = px.scatter(
        papers_per_period,
        x="period",
        y="count",
        trendline="ols",
        trendline_color_override="red",
        trendline_options=dict(log_y=True),
    )

    # Create a scatter plot with a logarithmic scale
    # To enable the ols trendline to depict a linear trend on a log scale log_y is set to True for both the plot and the trendline
    fig_log = px.scatter(
        papers_per_period,
        x="period",
        y="count",
        trendline="ols",
        trendline_color_override="red",
        log_y=True,
        trendline_options=dict(log_y=True),
    )

    # Combine the two plots into a single figure to display them side-by-side
    subfig = make_subplots(rows=1, cols=2, subplot_titles=["Linear Scale", "Log Scale"])

    # Add linear plot to the left side of the figure and log plot to the right side
    for trace in fig_linear.data:
        subfig.add_trace(trace, row=1, col=1)
    for trace in fig_log.data:
        subfig.add_trace(trace, row=1, col=2)

    # Update layout for the combined figure
    subfig.update_yaxes(
        title="# of Publications per Month", row=1, col=1, type="linear"
    )  # Update the y-axis label and linear-scale for the left plot
    subfig.update_yaxes(
        title="# of Publications per Month", row=1, col=2, type="log"
    )  # Update the y-axis label and set logarithmic-scale for the right plot
    subfig.update_xaxes(
        title="Year of Publication"
    )  # Update the x-axis label for both plots

    # Update the title text of the combined figure and center it
    subfig.update_layout(
        title=dict(
            text=f"Number of {title} Publications per Month",
            font=dict(size=TITLE_SIZE),
            x=TITLE_X_POS,
        )
    )

    subfig.show()


In [ ]:
plot_general_development(df_complete, "all", "M")

The growth of research in artificial intelligence and machine learning over the past three decades is reflected in the exponential rise in publications per month. As illustrated in the chart above, the number of publications has increased significantly, particularly from the early 2010s onward. This surge coincides with critical advancements in computational tools, such as open-source frameworks like TensorFlow and PyTorch, which democratized access to AI technologies and significantly reduced barriers to entry for researchers and developers.

On the linear scale, the data indicates a significant upward trend in the number of publications per month, with particularly accelerated growth in recent years. This increase can be attributed to the widespread adoption of AI technologies and the enhanced accessibility of research dissemination via platforms such as arXiv. These platforms, by enabling the rapid sharing of findings, have significantly contributed to the acceleration of innovation within the field. A logarithmic scale analysis further supports these observations, revealing a near linear trend indicative of sustained exponential growth. When viewed in conjunction with the conference trends, the publication data highlights a broader narrative: AI and ML have evolved from specialized academic disciplines into central drivers of scientific discovery and technological progress. This growth not only underscores the expanding influence of these fields but also signals their crucial role in shaping the future of research and innovation.

This trend is mirrored in the competitive landscape of AI and ML conferences, as shown in the chart below. Over the last decade, submissions to leading conferences such as NeurIPS, CVPR, and AAAI have risen exponentially, with counts reaching more than ten thousand annually. However, acceptance rates have concurrently declined, often falling below 30%, highlighting the increasing rigor and selectivity of these prestigious venues. This dual trend, rising submissions alongside decreasing acceptance rates, not only reflects the growing interest in these fields but also underscores their role in shaping the forefront of scientific discovery and technological progress.

![conference_trends](https://github.com/lixin4ever/Conference-Acceptance-Rate/blob/2cf5bf4d2801bf691857f56cec03c05cc99de29a/conference_trends.png?raw=true)
Source: [Github lixin4ever/Conference-Acceptance-Rate](https://github.com/lixin4ever/Conference-Acceptance-Rate/blob/2cf5bf4d2801bf691857f56cec03c05cc99de29a/conference_trends.png)

The trends in initial (*v1*) and corrective publications provide additional insights into the evolution of research within AI and ML. The charts below illustrate the number of publications per month for both categories, distinguishing between the introduction of new ideas and subsequent refinements.

Initial publications (*v1*) are fundamental to scientific progress, as they introduce novel methodologies, findings, or concepts. The data reveals that the growth in initial publications not only follows an exponential pattern but surpasses it. This indicates a phase of extraordinary acceleration in AI and ML research, driven by factors such as increased investments, widespread access to computational tools, and the practical integration of AI across various industries. The linear scale highlights the dramatic surge in initial contributions in recent years, while the logarithmic scale demonstrates that this growth significantly outpaces the expected trajectory of an exponential trend. This suggests that the pace of innovation in AI and ML is accelerating beyond conventional projections.

Corrective publications, encompassing versions beyond v1, represent the iterative nature of scientific research. These versions are typically created to address minor errors, refine methodologies, or clarify previously published findings. While essential to maintaining the quality and rigor of scientific work, corrective publications do not exhibit the same remarkable growth trends as initial publications. Their trajectory, as seen on a linear scale, reflects a steady and proportional increase over time, mirroring the overall expansion of research activity in the field. On a logarithmic scale, the trend in corrective publications suggests that while revisions are becoming more frequent, they remain supplementary to the primary contributions of initial publications. Corrective versions serve as a means to enhance the precision and reliability of the scientific record but do not fundamentally alter the state of knowledge or introduce groundbreaking ideas. Consequently, they are less indicative of the field's innovation trajectory and more reflective of its maturing standards of scholarly scrutiny.

Given the close alignment between the trends in initial and corrective publications, and the super-exponential growth observed in initial contributions, the analysis places its primary emphasis on the dataset filtered for v1 publications. This focus allows for a deeper exploration of the foundational contributions driving AI and ML research, avoiding the potential distortions introduced by iterative revisions. By honing in on initial publications, the analysis seeks to capture the most impactful advancements and innovations shaping the field.

In [ ]:
df_initial = df_complete[df_complete["id"].str.endswith("v1")]
plot_general_development(df_initial, "initial", "M")

In [ ]:
df_corrective = df_complete[~df_complete["id"].str.endswith("v1")]
plot_general_development(df_corrective, "corrective", "M")

## Trend analysis for the different Categories

In [31]:
def plot_line_chart_by_categories(df, title):
    """
    Plots line charts for the number of publications per year by category on both linear and logarithmic scales
    """
    # Generate consistent color map for categories (lines for same category have same color in both plots)
    color_map = generate_color_map(CATEGORIES)

    # Create a figure with two subplots: one for linear scale, one for log scale
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Linear-Scale", "Log-Scale"],
        shared_yaxes=False
    )

    # Loop through each category and add its data as a trace to the subplots
    for category in CATEGORIES:
        # Filter data for the current category
        category_data = df[df["categories"] == category]

        # Create a line trace for the current category
        trace = go.Scatter(
            x=category_data["period"],
            y=category_data["count"],
            mode="lines+markers",
            name=category,
            line=dict(color=color_map[category])
        )

        fig.add_trace(trace, row=1, col=1)  # Linear scale
        fig.add_trace(trace.update(showlegend=False), row=1, col=2)  # Log scale

    # Update layout for the figure
    fig.update_layout(
        title={
            "text": f"Number of {title} Publications per Year by Category",
            "font": {"size": TITLE_SIZE},
            "x": TITLE_X_POS,
        },
        xaxis_title="Year of Publication",  # X-axis label for left (linear) plot
        yaxis_title="# of Publications per Year",  # Y-axis label for left (linear) plot
        xaxis2_title="Year of Publication",  # X-axis label for right (log) plot
        yaxis2_title="# of Publications per Year",  # Y-axis label for right (log) plot
        yaxis2_type="log",  # Set log scale for right plot
        legend_title="Categories",  # Title for the legend
        height=550
    )
    fig.show()


def plot_heatmap_by_category(df, title):
    """
    Plots a heatmap showing the number of publications per year by category
    """
    # Create a pivot table to reshape the data for the heatmap
    # Rows represent categories, columns represent periods, values are publication counts
    pivot_counts = df.pivot(index="categories", columns="period", values="count").fillna(0).sort_index(ascending=False)  # Sort descending for categories to be sorted alphabetically

    # Define constants for visualizations
    colorscale = "RdBu_r"

    fig = go.Figure(
        data=go.Heatmap(
            z=pivot_counts.values,  # Publication counts
            x=pivot_counts.columns,  # Years
            y=pivot_counts.index,  # Categories
            text=np.array(pivot_counts.values).astype(int),  # Add exact counts as text
            texttemplate="%{text}",  # Display counts inside cells
            colorbar={"title": "Count"},  # Title for the color bar
            colorscale=colorscale  # Color scale for the heatmap
        )
    )

    # Update layout for heatmap
    fig.update_layout(
        title={
            "text": f"Number of {title} Publications per Year by Category",
            "font": {"size": TITLE_SIZE},
            "x": TITLE_X_POS,
        },
        xaxis_title="Year of Publication",  # X-axis label
        yaxis_title="Category",  # Y-axis label
        height=800
    )
    fig.show()

In [32]:
def visualize_publications_by_category(df, title):
    """
    Visualizes publications by category using line plots and a heatmap.
    """
    df = prepare_data(df, "Y", True)

    plot_line_chart_by_categories(df, title)
    plot_heatmap_by_category(df, title)


In [ ]:
visualize_publications_by_category(df_complete, "all")

#### Analysis of the exponential Trend Contribution by Category

The overall exponential growth in AI-related publications can be broken down into the handpicked categories that have been combined to create a representative overview of the field of ML and AI. When these categories are visualized individually, it becomes obvious that all categories follow a trend of increasing numbers of publications per year (e.g., Log-Scale). The Linear-Scale makes it evident that the categories Machine Learning (cs.LG), Artificial Intelligence (cs.AI), Computer Vision (cs.CV), Computation and Language (cs.CL), and Robotics (cs.RO), in particular, have been key contributors to the exponential growth in publications.

One of the main reasons for this exponential growth, which becomes particularly visible from 2015 onwards, is the widespread access to open-source frameworks such as TensorFlow (launched in 2015) and PyTorch (2016). These frameworks democratized machine learning research, allowing researchers from all backgrounds to prototype and deploy AI models more effectively. Around the same time, companies like Google, Facebook, and Microsoft began deploying machine learning solutions across various industries, addressing applied research problems such as recommender systems, autonomous vehicles, and healthcare diagnostics. In 2017, Google further accelerated this trend with the publication of the Transformer architecture, which revolutionized Natural Language Processing (NLP) by enabling significant breakthroughs, including the development of large-scale language models.

In NLP, the advancements made through chatbot systems, such as OpenAI’s GPT-2 (2019), combined with open-source NLP tools like Hugging Face (launched in 2016), further simplified and accelerated research efforts. These developments allowed researchers to leverage pre-trained models and focus on fine-tuning them for specific tasks, reducing barriers to entry and increasing productivity in the field.

With the growing volume of research to advance ML systems, coupled with the increasing presence of AI in consumer products like personal assistants (e.g., Siri, Alexa, etc.), the ethical, safety, and fairness aspects of AI systems gained prominence as critical research areas. This type of research was also supported by government institutions such as DARPA, which initiated efforts toward Explainable AI (XAI) in 2016. These initiatives aimed to ensure that AI systems are interpretable, trustworthy, and aligned with human values, further fueling interest in the ethical dimensions of AI research.

The field of Computer Vision has been an essential factor in the exponential growth of papers published regarding AI and ML. Some of the most important breakthroughs that catalyzed this growth were the advances made through deep Convolutional Neural Networks (2011), and Generative Adversarial Networks (GANs, 2014). These advancements enabled unprecedented improvements in image recognition, generation, and object detection tasks. Combined with the availability of large, publicly accessible datasets like ImageNet, which provided critical benchmarks, these innovations enabled faster development of real-world applications. For instance, the automation of tasks like image-based quality control in manufacturing, surveillance systems, and medical imaging diagnostics became more feasible. The integration of computer vision with robotics further drove industrial use cases, leading to rapid adoption and applied research efforts.

This branch of research has enabled another key contributor to the overall exponential trend: robotics. Self-driving car technology, spearheaded by companies like Tesla since 2015, has been a major driver of applied robotics research. Additionally, robotics research in general advanced through the availability of affordable and sophisticated robotic platforms, such as those developed by Boston Dynamics, which became accessible to researchers around 2015. These platforms facilitated experimentation in areas such as human-robot interaction, warehouse automation, and advanced mobility solutions, further expanding the scope and impact of robotics research.



In [15]:
visualize_publications_by_category(df_initial, "initial")

Since the data in the previous plot included not only the initial publications (v1) of papers but also their subsequent revisions (v2, v3, etc.), this plot presents the cleaned dataset, where only the first version of each paper is considered. In essence, this plot highlights the pure scientific contributions made in the field of ML and AI over the years, free from the influence of corrective or updated publications.

While there is an expected drop in the absolute number of publications, the overall trend remains largely unchanged, with only minor variations in the progression of individual categories. The key categories identified in the previous chart—Machine Learning (cs.LG), Artificial Intelligence (cs.AI), Computer Vision (cs.CV), Computation and Language (cs.CL), and Robotics (cs.RO)—continue to have the most substantial influence on the overall trend, maintaining their dominant role in shaping the field.

The total number of AI and ML-related papers published in 2024 decreased from over 160,000 (when including revisions) to over 110,000 (when considering only initial publications). This refined view underscores the immense volume of original contributions while providing a more accurate representation of the field's scientific growth.

In the chart below, the volume of revisions made to initial publications per year is visualized, representing the "noise" surrounding the pure scientific contributions. These revisions highlight a critical aspect of the research process, as they often involve updates, corrections, or refinements of existing work. While they do not directly contribute to new scientific discoveries, they play an essential role in maintaining the accuracy and reliability of the research. However, these revisions also introduce a time cost, potentially reducing the capacity for researchers to produce new, "pure" scientific contributions. For this reason, revisions should still be taken into account when evaluating the overall trend in the "number of publications per month," as they indirectly influence the productivity and focus of the research community.

In [16]:
visualize_publications_by_category(df_corrective, "corrective")

### Analysis of the relative impact by Category for each Year since 1994

In [17]:
def visualize_publications_by_category_normalized(df, title):
    """
    Visualizes the normalized distribution of publications per year by category.

    This function creates a heatmap where the values are normalized ratios of
    publications within each category. It shows the relative contribution of each
    category to the total publications for that category.
    """
    df = prepare_data(df, "Y", True)

    # Create a pivot table with categorises as rows and periods as columns
    pivot_counts = df.pivot(index="categories", columns="period", values="count").fillna(0)

    # Normalize publication counts so that each column (year) sums to 1
    normalized_counts = pivot_counts.div(pivot_counts.sum(axis=0), axis=1)
    # Sort categories alphabetically in descending order to sort the heatmap alphabetically (it's reversed)
    normalized_counts = normalized_counts.sort_index(ascending=False)

    # Create the heatmap figure
    fig = go.Figure(
        data=go.Heatmap(
            z=normalized_counts.values,
            x=normalized_counts.columns,
            y=normalized_counts.index,
            colorbar=dict(title="Ratio"),
            colorscale='RdBu_r'
        )
    )

    fig.update_layout(
        title=dict(
            text=f"Normalized Number of {title} Publications per Year by Category",
            font=dict(size=TITLE_SIZE),
            x=TITLE_X_POS,
            xanchor="center"
        ),
        xaxis_title="Year of Publication",
        yaxis_title="Category",
        height=800,
    )

    fig.show()

In [18]:
visualize_publications_by_category_normalized(df_complete, "initial")

#### Analysis of the Shift from Theoretical Foundations to Applied Advancements in AI and ML

The exponential growth in AI-related publications reflects a significant shift from early theoretical foundations to more applied advancements. This transition is particularly visible when analyzing the contributions of the key categories—Machine Learning (cs.LG), Artificial Intelligence (cs.AI), Computation and Language (cs.CL), and Computer Vision (cs.CV). While theoretical work dominated the early years of AI and ML research, the increasing availability of computational resources and the rise of practical frameworks drove the rapid adoption of applied methods.

By analyzing the relative importance of each category by year (in number of publications), it becomes evident that two additional key contributors emerged during the early days: Quantum Algebra (math.QA), which was dominant between 1994 and 2005, and Probability (math.PR), which played a critical role from 2002 to 2015. Another important category during this transition period was Dynamical Systems (math.DS), which had significant contributions between 1999 and 2011. These mathematical disciplines provided the foundational tools and theoretical insights necessary for developing many of the algorithms and techniques that would later drive applied AI research. *Konkrete Beispiele*

This shift from theory to applied research is further evidenced by the increasing dominance of computer science-focused categories over time. The prominence of math-focused categories gradually declined as the focus moved towards more practical domains within computer science (cs). Notably, an exception to this trend is Computation and Language (cs.CL), which showed early importance from 1994 to 1998, indicating that some applied research in natural language processing was already underway during the foundational phase. This category resurfaced increasingly from 2016 to 2024, driven by groundbreaking innovations such as the Transformer architecture and pre-trained language models.

The shift from theoretical to applied AI and ML was not a complete departure from foundational research but rather a natural evolution. Theoretical advancements laid the groundwork, providing the mathematical and algorithmic tools that enabled the explosion of applied research. Today, the interplay between theory and application continues to drive innovation, with applied research dominating the publication landscape while theoretical work adapts to solve the emerging challenges posed by real-world applications.